# Task 1: Initial Exploration & Summary 

# Import datasets

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df1 = pd.read_csv('Data.csv')
df2 = pd.read_csv('material.csv')
df_original1 = df1.copy()
df_original2 = df2.copy()

In [ ]:
print(df1.head(10).to_string(index=False))

In [ ]:
print(df2.head(10).to_string(index=False))

# Identify the total number of materials and heat treatment types. 

In [ ]:
# Find distinct values in the 'material' column of df1
distinct_materials = df1['Material'].nunique()
print("Total number of materials in df1:", len(df1['Material']))
print("Total number of unique materials in df1:", distinct_materials)

distinct_heat_treatments = df1['Heat treatment'].nunique()
print("Total number of heat treatments in df1:", len(df1['Heat treatment']))
print("Total number of unique heat treatments in df1:", distinct_heat_treatments)

# Check for any missing or inconsistent data values

In [ ]:
from  data_quality_check import data_quality_check
data_quality_check(df1)

# Handle Missing values

In [ ]:
# Fill missing values for objects in df1

df1['Heat treatment'] = df1['Heat treatment'].fillna('Unknown')
df1['Desc'] = df1['Desc'].fillna('Unknown')

In [ ]:
# Handling anomalies in the 'Sy' column of df1
df1['Sy'] = df1['Sy'].str.replace('max', '', regex=False)
print("Unique values in 'Sy' column before cleaning:", df1['Sy'].unique())
# Change the data type of 'Sy' column to integer
df1['Sy'] = df1['Sy'].astype(int)


In [ ]:
df_original1['Sy'] = df_original1['Sy'].str.replace('max', '', regex=False)
print("Unique values in 'Sy' column before cleaning:", df_original1['Sy'].unique())
# Change the data type of 'Sy' column to integer
df_original1['Sy'] = df_original1['Sy'].astype(int)

In [ ]:
# Fill missing values with mean for numeric columns in df1
for col in df1.select_dtypes(include=['number']).columns:
    mean_value = df1[col].mean()
    df1[col] = df1[col].fillna(mean_value)

# Identify the column containing outliers

In [ ]:
numeric_columns = df1.select_dtypes(include=['number']).columns

outlier_columns = []
for col in numeric_columns:
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    if ((df1[col] < lower_bound) | (df1[col] > upper_bound)).any():
        outlier_columns.append(col)

print("Columns containing outliers:", outlier_columns)

print("Number of outliers in each column:")
for col in outlier_columns:
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = ((df1[col] < lower_bound) | (df1[col] > upper_bound)).sum()
    print(f"{col}: {outliers_count} outliers")

# Plot the columns containing outliers

In [ ]:
# plot the columns containing outliers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_outliers(df, columns):
    for col in columns:
        plt.figure(figsize=(5, 2))
        sns.boxplot(x=df[col], color='lightblue', flierprops=dict(markerfacecolor='red', marker='o'))
        
        # Calculate statistics
        mean_val = df[col].mean()
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Add text labels for Q1, Q3, mean, lower bound, and upper bound
        stats_text = (f"Mean: {mean_val:.2f}\n"
                      f"25th Percentile (Q1): {Q1:.2f}\n"
                      f"75th Percentile (Q3): {Q3:.2f}\n"
                      f"Lower Bound: {lower_bound:.2f}\n"
                      f"Upper Bound: {upper_bound:.2f}")
        plt.gcf().text(0.85, 0.83, stats_text, fontsize=8, color='black', 
                       ha='right', va='top', bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))
        
        plt.title(f'Boxplot of {col}')
        plt.xlabel(col)
        plt.show()

plot_outliers(df1, outlier_columns)


In [ ]:
# Handle outliers by capping them at lower_bound and upper_bound
for col in outlier_columns:
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df1[col] = df1[col].clip(lower=lower_bound, upper=upper_bound)

# Plot comparison between cleaned data and original data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

def plot_comparison(df_original1, df1, columns):
    # Determine the number of rows and columns for the grid
    num_columns = 2  # Number of plots per row
    num_rows = len(columns)  # One row per column

    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))

    for i, col in enumerate(columns):
        # Plot for df_original1
        sns.histplot(df_original1[col], kde=True, ax=axes[i, 0], color='blue')
        mean_val = df_original1[col].mean()
        std_val = df_original1[col].std()
        max_val = df_original1[col].max()
        min_val = df_original1[col].min()
        
        # Add vertical lines for mean and standard deviation
        axes[i, 0].axvline(mean_val, color='red', linestyle='--', label='Mean')
        axes[i, 0].axvline(mean_val + std_val, color='green', linestyle='--', label='Mean + Std')
        axes[i, 0].axvline(mean_val - std_val, color='green', linestyle='--', label='Mean - Std')
        
        # Add text box with statistics
        stats_text = f"Mean: {mean_val:.2f}\nStd: {std_val:.2f}\nMax: {max_val:.2f}\nMin: {min_val:.2f}"
        axes[i, 0].text(0.95, 0.95, stats_text, transform=axes[i, 0].transAxes, fontsize=10,
                        verticalalignment='top', horizontalalignment='right',
                        bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))
        
        axes[i, 0].set_title(f'{col} (df_original1)')
        axes[i, 0].set_xlabel(col)
        axes[i, 0].set_ylabel('Frequency')

        # Plot for df1
        sns.histplot(df1[col], kde=True, ax=axes[i, 1], color='green')
        mean_val_orig = df1[col].mean()
        std_val_orig = df1[col].std()
        max_val_orig = df1[col].max()
        min_val_orig = df1[col].min()
        
        # Add vertical lines for mean and standard deviation
        axes[i, 1].axvline(mean_val_orig, color='red', linestyle='--', label='Mean')
        axes[i, 1].axvline(mean_val_orig + std_val_orig, color='green', linestyle='--', label='Mean + Std')
        axes[i, 1].axvline(mean_val_orig - std_val_orig, color='green', linestyle='--', label='Mean - Std')
        
        # Add text box with statistics
        stats_text_orig = f"Mean: {mean_val_orig:.2f}\nStd: {std_val_orig:.2f}\nMax: {max_val_orig:.2f}\nMin: {min_val_orig:.2f}"
        axes[i, 1].text(0.95, 0.95, stats_text_orig, transform=axes[i, 1].transAxes, fontsize=10,
                        verticalalignment='top', horizontalalignment='right',
                        bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))
        
        axes[i, 1].set_title(f'{col} (df1)')
        axes[i, 1].set_xlabel(col)
        axes[i, 1].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

# Call the function with the numeric columns
plot_comparison(df_original1, df1, numeric_columns)
#plot_comparison(df_original1, df1, ['Su', 'Sy', 'A5'])

# Check if the file exists, remove it if it does, and then save the cleaned DataFrame to a new CSV file
if os.path.exists('Cleaned_Data.csv'):
    os.remove('Cleaned_Data.csv')

df1.to_csv('Cleaned_Data.csv', index=False)


# Task 2: Groupwise Comparison

# Group average strength, ductility, and hardness values by Material type

In [ ]:
# Group average strength, ductility, and hardness values by Material type

df_grouped_Material = df1.groupby('Material').agg({
    'Su': 'mean',
    'Sy': 'mean',
    'A5': 'mean',
    'E': 'mean',
    'G': 'mean',
    'HV': 'mean'
}).reset_index()
print(df_grouped_Material.to_string(index=False))

df_grouped_Heat = df1.groupby('Heat treatment').agg({
    'Su': 'mean',
    'Sy': 'mean',
    'A5': 'mean',
    'E': 'mean',
    'G': 'mean',
    'HV': 'mean'
}).reset_index()
print(df_grouped_Heat.to_string(index=False))

# Comparing Efficiency with Material and Heat treatment as seperate metrics

In [ ]:
import seaborn as sns

# Define an efficiency metric (e.g., weighted average of properties)
df_grouped_Material['Efficiency'] = (df_grouped_Material['Su'] + df_grouped_Material['Sy'] + df_grouped_Material['A5'] + df_grouped_Material['E'] + df_grouped_Material['G'] + df_grouped_Material['HV']) / 6
df_grouped_Heat['Efficiency'] = (df_grouped_Heat['Su'] + df_grouped_Heat['Sy'] + df_grouped_Heat['A5'] + df_grouped_Heat['E'] + df_grouped_Heat['G'] + df_grouped_Heat['HV']) / 6

# Sort by efficiency
df_grouped_sorted_Material = df_grouped_Material.sort_values(by='Efficiency', ascending=False)
df_grouped_sorted_Heat = df_grouped_Heat.sort_values(by='Efficiency', ascending=False)

# Modify the bar plots to use logarithmic scales for better visualization of differences
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot the efficiency of each material type using a horizontal bar chart with text values
sns.barplot(y='Material', x='Efficiency', data=df_grouped_sorted_Material.head(5), hue='Material', palette='viridis', ax=axes[0], dodge=False)
axes[0].set_title('Top 5 Material Types by Efficiency')
axes[0].set_xlabel('Efficiency')
axes[0].set_ylabel('Material Type')
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.2f', label_type='center', fontsize=8)

# Plot the efficiency of each heat treatment type using a horizontal bar chart with text values
sns.barplot(y='Heat treatment', x='Efficiency', data=df_grouped_sorted_Heat.head(5), hue='Heat treatment', palette='coolwarm', ax=axes[1], dodge=False)
axes[1].set_title('Top 5 Heat Treatment Types by Efficiency')
axes[1].set_xlabel('Efficiency')
axes[1].set_ylabel('Heat Treatment Type')
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.2f', label_type='center', fontsize=8)

plt.tight_layout()
plt.show()

# Group average strength, ductility, and hardness values by Material type and Heat treatment

In [ ]:
# Group average strength, ductility, and hardness values by Material type
# Group by both 'Material' and 'Heat treatment' and calculate mean for other columns
df_grouped_strength = df1.groupby(['Material', 'Heat treatment']).agg({
    'Su': 'mean',
    'Sy': 'mean',
    'E': 'mean',
    'G': 'mean'
}).reset_index()

df_grouped_ductility = df1.groupby(['Material', 'Heat treatment']).agg({
    'A5': 'mean'
}).reset_index()

df_grouped_hardness = df1.groupby(['Material', 'Heat treatment']).agg({
    'Bhn': 'mean', 
    'HV': 'mean'
}).reset_index()

print(df_grouped_strength.to_string(index=False))
print(df_grouped_ductility.to_string(index=False))
print(df_grouped_hardness.to_string(index=False))


# Plot average strength, ductility, and hardness values by Material type and Heat treatment

In [ ]:
import seaborn as sns

# Define an efficiency metric (e.g., weighted average of properties)
df_grouped_strength['Efficiency'] = (df_grouped_strength['Su'] 
                                     + df_grouped_strength['Sy'] 
                                     + df_grouped_strength['E'] 
                                     + df_grouped_strength['G']) / 4
df_grouped_ductility['Efficiency'] = df_grouped_ductility['A5'] / 1
df_grouped_hardness['Efficiency'] = (df_grouped_hardness['Bhn'] 
                                     + df_grouped_hardness['HV']) / 2 

# Sort by efficiency
df_grouped_sorted_strength = df_grouped_strength.sort_values(by='Efficiency', ascending=False)
df_grouped_sorted_ductility = df_grouped_ductility.sort_values(by='Efficiency', ascending=False)
df_grouped_sorted_hardness = df_grouped_hardness.sort_values(by='Efficiency', ascending=False)

# Create subplots for strength, ductility, and hardness grouped by both Material and Heat treatment
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot the efficiency of each material and heat treatment type for strength
sns.barplot(y='Material', x='Efficiency', hue='Heat treatment', data=df_grouped_sorted_strength.head(5), palette='viridis', ax=axes[0], dodge=False)
axes[0].set_title('Top 5 Material and Heat Treatment Types by Strength Efficiency')
axes[0].set_xlabel('Efficiency')
axes[0].set_ylabel('Material Type')
axes[0].legend(title='Heat Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.2f', label_type='center', fontsize=8)

# Plot the efficiency of each material and heat treatment type for ductility
sns.barplot(y='Material', x='Efficiency', hue='Heat treatment', data=df_grouped_sorted_ductility.head(5), palette='coolwarm', ax=axes[1], dodge=False)
axes[1].set_title('Top 5 Material and Heat Treatment Types by Ductility Efficiency')
axes[1].set_xlabel('Efficiency')
axes[1].set_ylabel('Material Type')
axes[1].legend(title='Heat Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.2f', label_type='center', fontsize=8)

# Plot the efficiency of each material and heat treatment type for hardness
sns.barplot(y='Material', x='Efficiency', hue='Heat treatment', data=df_grouped_sorted_hardness.head(5), palette='magma', ax=axes[2], dodge=False)
axes[2].set_title('Top 5 Material and Heat Treatment Types by Hardness Efficiency')
axes[2].set_xlabel('Efficiency')
axes[2].set_ylabel('Material Type')
axes[2].legend(title='Heat Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
for container in axes[2].containers:
    axes[2].bar_label(container, fmt='%.2f', label_type='center', fontsize=8)

plt.tight_layout()
plt.show()
    

In [ ]:
print("Top 5 Material Efficiency:")
print(df_grouped_sorted_Material.head(5).to_string(index=False))

print("\nTop 5 Heat Treatment Types Efficiency:")
print(df_grouped_sorted_Heat.head(5).to_string(index=False))

print("\nTop 5 Material and Heat Treatment Types by Strength Efficiency:")
print(df_grouped_sorted_strength.head(5).to_string(index=False))

print("\nTop 5 Material and Heat Treatment Types by Ductility Efficiency:")
print(df_grouped_sorted_ductility.head(5).to_string(index=False))

print("\nTop 5 Material and Heat Treatment Types by Hardness Efficiency:")
print(df_grouped_sorted_hardness.head(5).to_string(index=False))

# Task 3: Design Ratio Analysis

In [ ]:
import pandas as pd

# Calculate the custom metrics
df1['Strength_Hardness'] = df1['Su'] / df1['Bhn']           
df1['Strength_Ductility'] = df1['Su'] * df1['A5']           
df1['Strength_Weight'] = df1['Su'] / df1['Ro']              

# Rank the materials based on the custom metrics
df1['Strength_Hardness_Rank'] = df1['Strength_Hardness'].rank(ascending=False, method='dense')
df1['Strength_Ductility_Rank'] = df1['Strength_Ductility'].rank(ascending=False, method='dense')
df1['Strength_Weight_Rank'] = df1['Strength_Weight'].rank(ascending=False, method='dense')

# Save the DataFrame with custom metrics to a new CSV file
# delete the csv file if it exists
import os
if os.path.exists('Data_with_custom_metrics.csv'):
    os.remove('Data_with_custom_metrics.csv')

df1.to_csv('Data_with_custom_metrics.csv', index=False)


In [ ]:
print("Top 5 Materials by Strength-to-Hardness Ratio:")

strength_hardness_table = df1[['Material', 'Strength_Hardness', 'Strength_Hardness_Rank']].sort_values(by='Strength_Hardness_Rank').head(5)
if os.path.exists('Top_5_Strength_Hardness.csv'):
    os.remove('Top_5_Strength_Hardness.csv')
strength_hardness_table.to_csv('Top_5_Strength_Hardness.csv', index=False)
print(strength_hardness_table.to_string(index=False))

print("\nTop 5 Materials by Strength-to-Ductility Index:")

strength_ductility_table = df1[['Material', 'Strength_Ductility', 'Strength_Ductility_Rank']].sort_values(by='Strength_Ductility_Rank').head(5)
if os.path.exists('Top_5_Strength_Ductility.csv'):
    os.remove('Top_5_Strength_Ductility.csv')
strength_ductility_table.to_csv('Top_5_Strength_Ductility.csv', index=False)
print(strength_ductility_table.to_string(index=False))

print("\nTop 5 Materials by Strength-to-Weight Proxy:")

strength_weight_table = df1[['Material', 'Strength_Weight', 'Strength_Weight_Rank']].sort_values(by='Strength_Weight_Rank').head(5)
if os.path.exists('Top_5_Strength_Weight.csv'):
    os.remove('Top_5_Strength_Weight.csv')
strength_weight_table.to_csv('Top_5_Strength_Weight.csv', index=False)
print(strength_weight_table.to_string(index=False))

In [ ]:
# plot 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Plot Strength-to-Hardness Ratio as a heatmap
sns.heatmap(strength_hardness_table.pivot_table(index='Material', values='Strength_Hardness'), 
            annot=True, fmt=".2f", cmap='Blues', ax=axes[0], cbar=False)
axes[0].set_title('Top 5 Materials by Strength-to-Hardness Ratio')
axes[0].set_xlabel('Strength-to-Hardness Ratio')
axes[0].set_ylabel('Material')

# Plot Strength-to-Ductility Index as a heatmap
sns.heatmap(strength_ductility_table.pivot_table(index='Material', values='Strength_Ductility'), 
            annot=True, fmt=".2f", cmap='Greens', ax=axes[1], cbar=False)
axes[1].set_title('Top 5 Materials by Strength-to-Ductility Index')
axes[1].set_xlabel('Strength-to-Ductility Index')
axes[1].set_ylabel('Material')

# Plot Strength-to-Weight Proxy as a heatmap
sns.heatmap(strength_weight_table.pivot_table(index='Material', values='Strength_Weight'), 
            annot=True, fmt=".2f", cmap='Reds', ax=axes[2], cbar=False)
axes[2].set_title('Top 5 Materials by Strength-to-Weight Proxy')
axes[2].set_xlabel('Strength-to-Weight Proxy')
axes[2].set_ylabel('Material')

# Adjust layout
plt.tight_layout()
plt.show()